In [ ]:
import asyncio
import random

from promisio import promisify
from dotenv import load_dotenv
from langchain.agents.tools import Tool
from langchain.chat_models import ChatOpenAI

from async_agent import AsyncChatAgentExecutor
from async_agent.structured_chat.base import StructuredChatAgent

load_dotenv()

In [ ]:
@promisify
async def tool_a(*args, **kwargs):
    await asyncio.sleep(10)
    return  str(random.randint(0, 100))


@promisify
async def tool_b(*args, **kwargs):
    await asyncio.sleep(10)
    return  "tool_b's result"


In [ ]:
tools = [
    Tool(
        name="tool_a", 
        func=tool_a,
        description="tool_a's description"
    ),
    Tool(
        name="tool_b",
        func=tool_b,
        description="tool_b's description"
    )
]

chat = ChatOpenAI()

agent = StructuredChatAgent.from_llm_and_tools(
    llm=chat,
    tools=tools,
)

executor = AsyncChatAgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,   
)


In [ ]:
executor({
    "input": "Run tool_a",
})

In [8]:
agent

StructuredChatAgent(llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, prompt=ChatPromptTemplate(input_variables=['input', 'agent_scratchpad'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\ntool_a: tool_a\'s description, args: {{{{\'tool_input\': {{{{\'type\': \'string\'}}}}}}}}\ntool_b: tool_b\'s description, args: {{{{\'tool_input\': {{{{\'type\': \'string\'}}}}}}}}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or tool_a, tool_b\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nTh